# Encoding comparison

This notebook contains a side-by-side evaluation of different bit-to-tree encodings

In [44]:
import sys
sys.path.append('../')
sys.setrecursionlimit(10000) # Croissant

import numpy as np
import math
import plotly.express as px
import pandas as pd
from func_timeout import func_timeout, FunctionTimedOut
import traceback

from tree_lib.tree import TreeNode, tower
from tree_lib.encodings import two_choices, two_choices_one_two, counting_ones, counting_ones_leaves, fibonaccio, tuples, square, cube, entropy, power_digits, power_digits_2, gauss, cantor2d, simplex, one_child, abe1993
import tree_lib.util as util

#### 1 - Define encodings to compare

In [45]:

ENCODINGS_UNFILTERED = [
    #### Display  | bits to tree | tree to bits | max bits encodable (None if infinite)
    # ["n*(n+1)/2", gauss.bits_to_tree, gauss.tree_to_bits,None],
    # ["two_choices", two_choices.bits_to_tree, two_choices.tree_to_bits,None],
    # ["two_choices_one_two", two_choices_one_two.bits_to_tree, two_choices_one_two.tree_to_bits,None],
    # ["two_choices_wrap", two_choices.bits_to_tree_wrap, None,None],
    # ["two_choices_mark", two_choices.bits_to_tree_mark, None,None],
    # ["two_choices_k=9", lambda s: two_choices.bits_to_tree_k(s, 9), None,None],
    # ["counting_ones_leaves", counting_ones_leaves.bits_to_tree, counting_ones_leaves.tree_to_bits,None],
    # ["fibonaccio", fibonaccio.bits_to_tree, fibonaccio.tree_to_bits,None],
    # ["power_digits", power_digits.bits_to_tree, power_digits.tree_to_bits,None],
    # ["power_digits_2", power_digits_2.bits_to_tree, power_digits_2.tree_to_bits,None],
    # ["tuples", tuples.bits_to_tree, tuples.tree_to_bits,None],
    ["Y.Abe, 1994", abe1993.bits_to_tree, abe1993.tree_to_bits,31],
    ["Squares", square.bits_to_tree, square.tree_to_bits,None],
    ["Cubes", cube.bits_to_tree, cube.tree_to_bits,None],
    # ["entropy", entropy.bits_to_tree, entropy.tree_to_bits,None],
    # ["cantor2d", cantor2d.bits_to_tree, cantor2d.tree_to_bits,None],
    # ["cantor2d_snake", lambda s: cantor2d.bits_to_tree(s, snake=True), lambda t: cantor2d.tree_to_bits(t, snake=True),None],
    # ["simplex2d", lambda s: simplex.bits_to_tree(s, 2), lambda t: simplex.tree_to_bits(t, 2),None],
    # ["simplex3d", lambda s: simplex.bits_to_tree(s, 3), lambda t: simplex.tree_to_bits(t, 3),None],
    # ["one_child", one_child.bits_to_tree, one_child.tree_to_bits,None],
    # ["simplex4d", lambda s: simplex.bits_to_tree(s, 4), lambda t: simplex.tree_to_bits(t, 4),None],
    # ["simplex5d", lambda s: simplex.bits_to_tree(s, 5), lambda t: simplex.tree_to_bits(t, 5),None],
    # ["simplex10d", lambda s: simplex.bits_to_tree(s, 10), lambda t: simplex.tree_to_bits(t, 10),None],
    ]

#### 2. Check for correctness

In [46]:


from func_timeout import func_timeout, FunctionTimedOut

BASIC_STRINGS = [
    "0","1","01","10","11", "100", "101", "110", "111",     # basic
    "010101010101", "0000000000", "1111111111",             # something more complex
]
LONG_STRINGS = [util.gen_bit_string(i) for i in range(1, 400)]

def string_test(strings, name, btt, ttb, max_bits, shuffle=False):
    def encode_decode(s):
        t = btt(s)
        if t is not None and shuffle:
            util.shuffle_tree(t)
        return ttb(t)
    
    # Filter out strings for encodings that have limited capabilities
    if max_bits is not None:
        orig_strings_len = len(strings)
        strings = [s for s in strings if len(s) < max_bits]
        print(f"{name}: Keeping only {len(strings)}/{orig_strings_len} strings -- length <= {max_bits}")

    try:
        if ttb is None:
            # If there is no ttb, try to encode strings only
            res = [btt(s) for s in strings]
            return f"🤷️️️️️️"
        else:
            errors = func_timeout(10., lambda: [s for s in strings if encode_decode(s) != s])
            return "✅" if len(errors)==0 else f"🐖️️️️️️ {errors[0]}"
    except FunctionTimedOut:
        return f"⏰"
    except Exception as e:
        print(traceback.format_exc())
        return f"🧨️️️️️️ {e}"

results = [[name, 
            string_test(BASIC_STRINGS, name, btt, ttb, max_bits), 
            string_test(LONG_STRINGS, name, btt, ttb, max_bits),
            string_test(BASIC_STRINGS+LONG_STRINGS, name, btt, ttb, max_bits, True),] 
            for name, btt, ttb, max_bits in ENCODINGS_UNFILTERED]

df = pd.DataFrame(results, columns=['Name', 'Basic Test', "Long Strings", "Unordered"])

# Discard encodings that do not work
def is_acceptable_result(test_basic,test_long,test_unordered):
    return ("✅" in test_basic or "🤷️️️️️" in test_basic) and \
           ("✅" in test_long or "🤷️️️️️" in test_long)

to_discard = [name for [name, t1, t2, t3] in results if not is_acceptable_result(t1,t2,t3)]
if to_discard:
    print(f"\nDiscarding {to_discard}")
ENCODINGS = [e for e in ENCODINGS_UNFILTERED if e[0] not in to_discard]

df

Y.Abe, 1994: Keeping only 12/12 strings -- length <= 31
Y.Abe, 1994: Keeping only 30/399 strings -- length <= 31
Y.Abe, 1994: Keeping only 42/411 strings -- length <= 31


,Name,Basic Test,Long Strings,Unordered
0,"Y.Abe, 1994",✅,✅,✅
1,Squares,✅,✅,✅
2,Cubes,✅,✅,✅


#### 3. Generate dataset


In [47]:
BIT_STRING_LENGTHS = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
PROBABILITY_OF_ONE = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
AUTOCORRELATION = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
SAMPLES_PER_STRING_TYPE = 100
SAMPLES_PER_STRING_TYPE_AC = 3

# Build generic random data set with random string. 
dataset_generic = [util.gen_bit_string(length, prob_of_one) 
                        for length in BIT_STRING_LENGTHS 
                        for prob_of_one in PROBABILITY_OF_ONE
                        for _ in range(SAMPLES_PER_STRING_TYPE)]

# Inject some edge cases (all zeros, all ones)
dataset_generic += [util.gen_bit_string(length, prob_of_one) 
                        for length in BIT_STRING_LENGTHS 
                        for prob_of_one in [0., 1.]]    


dataset_generic += [util.gen_bit_string_with_autocorrelation(length, ac) 
                        for length in BIT_STRING_LENGTHS 
                        for ac in AUTOCORRELATION
                        for _ in range(SAMPLES_PER_STRING_TYPE_AC)]              

def dataset_for_encoding (encoding_name, bit_to_tree_fun, max_bits): 
    filtered_dataset = dataset_generic
    if max_bits is not None:
        orig_strings_len = len(filtered_dataset)
        filtered_dataset = [s for s in filtered_dataset if len(s) < max_bits]
        print(f"{encoding_name}: Keeping only {len(filtered_dataset)}/{orig_strings_len} strings -- length <= {max_bits}")

    # Generate random bit strings with various properties
    # Given a bit string, generate a data point
    # [encoding name, length, num nodes]
    def data_point(bit_str):
        tree = bit_to_tree_fun(bit_str)
        str_len = len(bit_str)
        perc_of_ones = bit_str.count('1') / str_len
        autocorrelation = util.autocorrelation(bit_str)
        return [encoding_name, str_len, perc_of_ones, autocorrelation, tree.n_descendants/str_len, tree.get_total_footprint()]
    
    return [data_point(bit_str) for bit_str in filtered_dataset]

dataset = []
for encoding_name, btt, _, max_bits in ENCODINGS:
    dataset += dataset_for_encoding(encoding_name, btt, max_bits)

df = pd.DataFrame(dataset, columns=['Encoding', 'Bits', 'PercOfOnes', 'Autocorrelation', 'Nodes/Bit', 'Footprint']) 
df


Y.Abe, 1994: Keeping only 2787/9290 strings -- length <= 31


,Encoding,Bits,PercOfOnes,Autocorrelation,Nodes/Bit,Footprint
0,"Y.Abe, 1994",10,0.10,0.80,1.20,30
1,"Y.Abe, 1994",10,0.20,0.60,1.10,29
2,"Y.Abe, 1994",10,0.20,0.90,1.10,36
3,"Y.Abe, 1994",10,0.10,0.80,1.20,29
4,"Y.Abe, 1994",10,0.30,0.50,1.10,34
...,...,...,...,...,...,...
21362,Cubes,100,0.46,0.80,1.83,693
21363,Cubes,100,0.54,0.86,1.97,739
21364,Cubes,100,0.54,0.83,1.94,739
21365,Cubes,100,0.46,0.90,1.90,724


In [57]:
grouped_df = df.groupby(['Encoding', 'Bits']).size().reset_index(name='Count')
grouped_df

grouped_stats = df.groupby(['Encoding', 'Bits'])['Footprint'].agg(
    Median='median',
    StdDev='std'
).reset_index()
grouped_stats

squares_df = df[df['Encoding'] == 'Squares'].groupby('Bits')['Footprint'].sum().reset_index(name='SquaresFootprint')
yabe_df = df[df['Encoding'] == 'Y.Abe, 1994'].groupby('Bits')['Footprint'].sum().reset_index(name='YabeFootprint')

# Merge the two DataFrames on "Bits"
merged_df = pd.merge(squares_df, yabe_df, on='Bits', how='inner')
merged_df['Ratio'] = merged_df['SquaresFootprint'] / merged_df['YabeFootprint']
merged_df

,Bits,SquaresFootprint,YabeFootprint,Ratio
0,10,43122,31133,1.385090
1,20,91650,62894,1.457214
2,30,158698,96017,1.652811


#### 4. Reports

In [48]:

fig = px.box(df, x="Bits", y="Nodes/Bit", color="Encoding", 
             title="Bits vs Number Nodes, Entire Dataset")
fig.show()

fig = px.box(df, x="Bits", y="Footprint", color="Encoding",
             title="Footprint Score vs Bits")
# fig.update_yaxes(tickmode="array", tickvals=[1, 10, 100, 1000, 10000], tickformat=",")
# fig.update_xaxes(tickmode="array", tickvals=BIT_STRING_LENGTHS, tickformat=",")
fig.show()

df_500 = df[df['Bits'] == 500] 
fig = px.scatter(df_500, x="PercOfOnes", y="Nodes/Bit", color="Encoding", 
             title="Percentage of Ones vs Num Nodes, 500 Bits")
fig.show()

fig = px.scatter(df_500, x="Autocorrelation", y="Nodes/Bit", color="Encoding", 
             title="Autocorrelation vs Num Nodes, 500 Bits")
fig.show()


# Number-Based Evaluation

Some encodings are sensitive to certain numbers, because they are easier to represent in that space. 
Evaluate a contiguous number in a range.

In [49]:
def plot_range(title, center, interval_size, n_samples):
    strings = [(i,util.number_to_bit_string(i)) for i in range(center-interval_size//2,
                                                               center+interval_size//2, 
                                                               interval_size//n_samples)]
    dataset_scatter = []
    for encoding_name, btt, _, _ in ENCODINGS:
        dataset_scatter += [[encoding_name, i+1, btt(s).n_descendants] for i,s in strings]

    dfs = pd.DataFrame(dataset_scatter, columns=['Encoding', 'Number', 'Nodes']) 
    dfs

    fig = px.line(dfs, x="Number", y="Nodes", color="Encoding", 
                title=f"Number of Nodes in a Range of Numbers -- {title}")
    fig.show()


plot_range("Range", 100002, 200000, 10000)


# Tower

We now compare the encodings in the "tower" edge case, that is, a tree that just grows vertically, having a single child each time.

In [50]:
dataset = []
for encoding_name, _, ttb in ENCODINGS:
    try:
        dataset += [[encoding_name, i, len(ttb(tower(i,2)))] for i in range(2,10)]
    except:
        print(f"{encoding_name} does not support tower")
        continue

dfs = pd.DataFrame(dataset, columns=['Encoding', 'Tower', 'Bits']) 
dfs

fig = px.line(dfs, x="Tower", y="Bits", color="Encoding", 
                title=f"Bits w.r.t. to tower height")
fig.show()

ValueError: too many values to unpack (expected 3)